In [ ]:
import pandas as pd
from sodapy import Socrata
import time
import sys
import json
import geopandas as gpd
import fiona
import shapely
import pylab as pl
import numpy as np
from shapely.geometry import asShape
import urllib.request, json 
from tqdm import tqdm
from tqdm import tqdm_notebook
from time import sleep

In [ ]:
client = Socrata("data.cityofnewyork.us", 'VQtPna4DyINdAHU8AgxFGwiEr')
def polygon_query_points_id(client, MTPLG):
    results = client.get("fhrw-4uyv", 
                         select = 'unique_key',
                         limit = 5000000000000,
                         where = "within_polygon(location, '{0}')".format(MTPLG))
    return results

Load census block shapefile from [link](https://data.cityofnewyork.us/City-Government/2010-Census-Blocks/v2h8-6mxf/data)

In [ ]:
cb_raw = client.get("twhy-dzjp", limit = 5000000000000)
cb = gpd.GeoDataFrame.from_records(cb_raw)
cb['geometry'] = list(map(lambda x: asShape(x), cb.the_geom))

In [ ]:
cb.head()

In [ ]:
def create_dict(i):
    results = polygon_query_points(client, cb['geometry'][i])
    results = [dict(item, bctcb2010 = cb['bctcb2010'][i]) for item in results]
    return results

In [ ]:
L = []
exceptions = []
for i in tqdm_notebook(range(0,38795-1000, 1000)):
    for j in tqdm_notebook(range(i, i + 1000)):
        try: 
            L = L + create_dict(j)
        except Exception:
            exceptions.append(j)

In [ ]:
 for j in tqdm_notebook(range(38000, 38794)):
        try: 
            L = L + create_dict(j)
        except Exception:
            pass

In [ ]:
pd.DataFrame(L).to_csv('cb_2010_311_partial.csv', index = False)

census tract [link](https://data.cityofnewyork.us/City-Government/2010-Census-Tracts/fxpq-c8ku/data)

In [ ]:
len(exceptions)

In [ ]:
max(cb.index)

In [ ]:
exceptions